In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import annotations

import random
import math
import abc
import tensorflow as tf
import numpy as np

from tf_agents import environments
from tf_agents import specs
from tf_agents import trajectories
from tf_agents import networks
from tf_agents import agents
from tf_agents import utils
from tf_agents import policies
from tf_agents import replay_buffers
from tf_agents import drivers
from tf_agents import typing


from InterruptEnvironment import InterruptEnvironment as InterruptEnv
from InterruptDriver import InterruptDriver
from TransitionBuffer import TransitionBuffer

In [2]:
learning_rate = 1e-3 
replay_buffer_max_length = 1000 
initial_collect_steps = 100 
batch_size = 4 

num_eval_episodes = 10 
collect_steps_per_iteration = 1
num_iterations = 5000

log_interval = 200
eval_interval = 1000

In [3]:
class SimulatorEnv(InterruptEnv):
  def __init__(self) -> None:
    env_name = 'CartPole-v0'
    self.env = environments.suite_gym.load(env_name)
    super().__init__()

  def observation_spec(self):
    return self.env.observation_spec()

  def action_spec(self):
    return self.env.action_spec()

  def _reset(self):
    time_step = self.env.reset()
    interrupt = InterruptEnv.Interrupt(time_step, actionRequired=True)
    self._counter = 0
    return interrupt

  def _resume(self, actionStep: trajectories.policy_step.PolicyStep = None):
    
    if (self.current_interrupt().actionRequired()):
      if (actionStep == None):
        raise ValueError("When interrupt is of type action, actionstep cannot be None")
      timeStep = self.current_interrupt().timeStep()
      nextTimeStep = self.env.step(actionStep.action)
      return InterruptEnv.Interrupt(timeStep, transitionCompleted=True,
                                    nextTimeStep=nextTimeStep, actionStep=actionStep)
    elif(self.current_interrupt().transitionCompleted()):
      self._counter += 1
      if (self._counter >= 1000000):
        return InterruptEnv.Interrupt(self.current_interrupt().nextTimeStep(), terminated=True)
      else:
        if (self.current_interrupt().nextTimeStep().step_type == trajectories.time_step.StepType.LAST):
          return InterruptEnv.Interrupt(self.env.reset(), actionRequired=True)
        else:
          return InterruptEnv.Interrupt(self.current_interrupt().nextTimeStep(), actionRequired=True)
    elif(self.current_interrupt().terminated()):
      raise RuntimeError("Simulation has already ended")

In [4]:
env = SimulatorEnv()
interrupt = env.reset()

In [5]:
env.time_step_tensor_spec()

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32), maximum=array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32)),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

# Agent

In [6]:
fc_layer_params = (50, 20)
action_tensor_spec = env.action_tensor_spec()
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None)
q_net = networks.sequential.Sequential(dense_layers + [q_values_layer])

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = agents.dqn.dqn_agent.DqnAgent(
    env.time_step_tensor_spec(),
    env.action_tensor_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=utils.common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

#Random Policy test ->

In [8]:
test_environment = SimulatorEnv()
random_policy = policies.random_tf_policy.RandomTFPolicy(test_environment.time_step_tensor_spec(),
                                                test_environment.action_tensor_spec())

# Creating objects from Custom Driver and Buffer

In [9]:
buffer = TransitionBuffer(5000)

In [10]:
TestDriver = InterruptDriver(
    env,
    policies.py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [buffer.observer])

## test

In [11]:
TestDriver.run(100)

()

In [12]:
buffer.sampleExperiences(5)

Trajectory(
{'action': <tf.Tensor: shape=(5, 2), dtype=int64, numpy=
array([[1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1]], dtype=int64)>,
 'discount': <tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>,
 'next_step_type': <tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1]])>,
 'observation': <tf.Tensor: shape=(5, 2, 4), dtype=float32, numpy=
array([[[-0.00793626, -1.0002933 ,  0.01219268,  1.3614848 ],
        [-0.02794212, -0.80532616,  0.03942237,  1.0726405 ]],

       [[ 0.0401236 ,  0.76591563, -0.05258388, -1.2232558 ],
        [ 0.05544192,  0.96167403, -0.077049  , -1.5319401 ]],

       [[-0.01334328, -0.5717279 , -0.01125514,  0.77151567],
        [-0.02477784, -0.7666932 ,  0.00417517,  1.0606362 ]],

       [[-0.13869052, -0.4373571 ,  0.18935208,  0.8485414 ],
        [-0.14743765, -0.2

#Train

In [13]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


In [14]:

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent._train = utils.common.function(agent._train)

# Reset the train step.
agent.train_step_counter.assign(0)


# Create a driver to collect experience.
collect_driver = InterruptDriver(
    env,
    policies.py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [buffer.observer])

for _ in range(5000):

  collect_driver.run(10)

  experience = buffer.sampleExperiences(200)
  train_loss = agent.train(experience, None).loss
  if (_ % 100 == 0):
    print(train_loss)
  if (_ % 500 == 0):
    print(compute_avg_return(environments.tf_py_environment.TFPyEnvironment(env.env), agent.policy))

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
tf.Tensor(1.8090191, shape=(), dtype=float32)
9.4
tf.Tensor(14.754783, shape=(), dtype=float32)
tf.Tensor(5.711733, shape=(), dtype=float32)
tf.Tensor(4.7247424, shape=(), dtype=float32)
tf.Tensor(1.35517, shape=(), dtype=float32)
tf.Tensor(1.2943013, shape=(), dtype=float32)
200.0
tf.Tensor(8.773051, shape=(), dtype=float32)
tf.Tensor(12.615079, shape=(), dtype=float32)
tf.Tensor(49.483307, shape=(), dtype=float32)
tf.Tensor(104.271194, shape=(), dtype=float32)
tf.Tensor(46.64486, shape=(), dtype=float32)
79.8
tf.Tensor(43.04545, shape=(), dtype=float32)
tf.Tensor(108.799866, shape=(), dtype=float32)
tf.Tensor(23.421986, shape=(), dtype=float32)
tf.Tensor(46.724796, shape=(), dtype=float32)
tf.Tensor(260.25555, shape=(), dtype=float32)
141.1
tf.Tensor(6.0

In [15]:
eval_env = environments.tf_py_environment.TFPyEnvironment(env.env)
compute_avg_return(eval_env, agent.policy)

200.0